# Tutorial Restoring Symmetry
In the tutorial *CanThisBeTheGroundState.ipynb* we found that the ground state of (mostly insulating) impurity problems can spontaneously break spin symmetry. We checked that this was not a convergence issue of DMRG but that this is what actually happens in these problems for any finite sized system.

Physically this is of course an indication that the underlying lattice model (if we think in terms of DMFT) has some tendency to become magnetic. In true single site DMFT we cannot a-priori say whether this is going to be Ferro- or Anti-Ferromagnetism, we only know something magnetic is happening.

The purpose of this tutorial is to show that the two ground-states really are symmetric partners and that we can extract one Green's function from the other by just flipping spins.

In [ ]:
#%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np

# ForkTPS
from forktps.DiscreteBath import *
from forktps.Helpers import *

from triqs.gf import *
from triqs.operators import *

# make IPython use more of the screen
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

def EmptySolver(bath, e0):
    S = ftps.Solver(gf_struct = gfstruct , nw = 3001, wmin=-10., wmax=10.)
    S.b, S.e0 = bath, e0
    
    return S

First, prepare the same system as in the tutorial *CanThisBeTheGroundState.ipynb*:

In [ ]:
Norbs = 1
gfstruct = [['up',[0]], ['dn',[0]]]

# Interaction
U = 6
# Bath size
Nb = 10


# Bath hybridization consists of two semi-circles at -3 and 3
delta = GfReFreq(indices = [0], window = (-10., 10.), n_points =  10001, name = 'up')
delta['0','0'] << SemiCircular(half_bandwidth = 1.0, chem_potential= -3) + SemiCircular(half_bandwidth = 1.0, chem_potential=3)
Delta = BlockGf(name_list = ('up','dn'), block_list = (delta,delta), make_copies = True)


# Discretize hybridization, gap tells the function to place no bath sites between -2 and 2
bath = DiscretizeBath(Delta, Nb=Nb, gap=[-2.,2.] )

# Calculate the discrete hybridization and compare it.
DeltaRe = bath.reconstructDelta( w = delta.mesh, eta = 0.1 )
plt.figure()
for j in range(1):
    plt.plot(getX(Delta.mesh), -Delta  ['up'][j,j].imag.data, label = 'Original indx ' + str(j))
    plt.plot(getX(Delta.mesh), -DeltaRe['up'][j,j].imag.data, label = 'Reconstruction indx ' + str(j))
    
    
e0 = ftps.solver_core.Hloc(gfstruct) #give the local Hamiltonian the right block structure
e0.Fill([[-U/2]]) # put the same 1x1 matrix on both blocks

# Interaction
Hint = ftps.solver_core.HInt(u=U, j=0., up=0., dd = False)

We know that the two ground states are in the sector [5,6] and [6,5] respectively. So solve the impurity problem in each sector and store the solvers:

In [ ]:
# Spin-up Polarized and Spin-Down Polarized
names = ['UpPol', 'DnPol']
Solvers = {names[0] : EmptySolver(bath, e0), names[1] : EmptySolver(bath, e0) }
NParts =  {names[0] : [[6,5]], names[1] : [[5,6]] }

# parameters and ground state measurements
tevo = ftps.solver.TevoParams(dt = 0.1, time_steps = 20)
dmrg = ftps.solver.DMRGParams(sweeps = 20, tw = 1E-12)
measurements = [n('up',0), n('dn',0)]

for name, NPart in NParts.items():
    print('\nCalculation',name,'in sector', *NPart)
    S = Solvers[name]
    
    S.solve( h_int = Hint, 
         tevo = tevo,  
         params_partSector = dmrg,
         params_GS = dmrg,
         NPart = NPart,
         measurements = measurements
         )
    
    # The result of the measurement can be found in S.measurment_results
    for indx, meas in enumerate(measurements):
        print(f'Measurement of Operator: {meas} resulted in {S.measurement_results[indx]:.3}' )
    print(f'Ground state energy: {S.E0}')

The output makes sense, the ground state impurity occupation in the two sectors is symmetric and the energy is the same. Let us check the Green's function:

In [ ]:
colors = {names[0]: 'red', names[1]: 'blue'}

plt.figure( figsize = [8,4] )

# Plot Green's functions themselves
plt.subplot(1,2,1)
plt.title('$G_{ret}(t)$')
for name, S in Solvers.items():
    Gret = S.G_ret

    plt.plot( getX(Gret.mesh), Gret['up'][0,0].real.data, 'x-',  color = colors[name], label = f'{name} $G_{{up}}$' )
    plt.plot( getX(Gret.mesh), Gret['dn'][0,0].real.data, 'o--', color = colors[name], label = f'{name} $G_{{dn}}$' )

plt.xlabel('t')
plt.legend()


# Plot difference between UpPol spin-up Green's function and DnPol spin-down Grenn's function (and vice versa)
plt.subplot(1,2,2)
plt.title('Symmetric Difference')
G1 = Solvers[names[0]].G_ret
G2 = Solvers[names[1]].G_ret

diff = np.abs( G1['up'][0,0].real.data - G2['dn'][0,0].real.data )
plt.plot( getX(Gret.mesh), diff, color = 'red' , label = '$|G_{1, up} - G_{2,dn}|$')

diff = np.abs( G1['dn'][0,0].real.data - G2['up'][0,0].real.data )
plt.plot( getX(Gret.mesh), diff, color = 'blue', label = '$|G_{1, dn} - G_{2, up}|$')
plt.legend()
plt.xlabel('t')



Indeed, these two Green's functions are symmetric pairs of each other and we can get on from the other by just flipping the spins. The question remains, what is now the proper Green's function?

If one wants to obtain a magnetic solution, this magnetic ground state is exactly what should be used. The paramagnetic case is quite subtle and should be discussed a little bit:<br><br>
If we denote $| 1 \rangle$ and $| 2 \rangle$ for the two different ground states, any linear combination $ c_1 | 1 \rangle + \sqrt{1-c_1^2} | 2 \rangle$ would be a valid ground state as would be the resulting Green's function. Still, there is one proper thermodynamic choice. The density matrix $\rho = Z^{-1} \lim_{\beta \to \infty} e^{-\beta H}$ populates each of the two degenerate ground states by an equal amount, so the only consistent paramagnetic choice would be $\frac{1}{\sqrt{2}} \left( | 1 \rangle + | 2 \rangle \right)$ (phases are unimportant see formula below).

The resulting Green's function is then:

$$ G_{Paramagnetic} = \frac{1}{2} \left( \langle 1 | \left\{c(t), c^\dagger \right\} |1\rangle + \langle 2 | \left\{ c(t), c^\dagger \right\} |2\rangle + \underbrace{ \langle 2 |  \left\{c(t), c^\dagger \right\} |1\rangle}_{=0} + \underbrace{\langle 1 | \left\{c(t), c^\dagger \right\} |2\rangle}_{=0} \right) $$

where the cross terms vanish because the two states are in different sectors of particle number. Hence we find the very intuitive result of just averaging the two Green's functions.

In [ ]:
G1 = Solvers[names[0]].G_ret
G2 = G1.copy()
G2['up'], G2['dn'] = G1['dn'], G1['up']

G_Paramagnetic = 0.5*( G1 + G2 ) 

A final remark: While Green's functions can be symmetrized this way, self energies cannot, because $ (G_1 + G_2)^{-1} \neq G_1^{-1} + G_2^{-1}$ and the symmetrization has to be done one the level of the Green's function i.e. _before_ the calculation of the self energy.